In [1]:
# import glob
# import os
# import sys
# import time

# try:
#     sys.path.append(glob.glob('../../carla/dist/carla-*%d.%d-%s.egg' % (
#         sys.version_info.major,
#         sys.version_info.minor,
#         'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
# except IndexError:
#     pass

# import carla

In [2]:
# def sensor_callback(data):
#     """
#     This simple callback just stores the data on a thread safe Python Queue
#     to be retrieved from the "main thread".
#     """
#     for location in data:
#         print(location.point.x)
        


In [3]:
# client = carla.Client('localhost', 2000)
# client.set_timeout(5.0)
# world = client.get_world()
# bp_lib = world.get_blueprint_library()

# # Search the desired blueprints
# vehicle_bp = bp_lib.filter("vehicle.lincoln.mkz2017")[0]
# camera_bp = bp_lib.filter("sensor.camera.rgb")[0]
# lidar_bp = bp_lib.filter("sensor.lidar.ray_cast")[0]

# lidar_bp.set_attribute('dropoff_general_rate', '0.0')
# lidar_bp.set_attribute('dropoff_intensity_limit', '1.0')
# lidar_bp.set_attribute('dropoff_zero_intensity', '0.0')


# # Spawn the blueprints
# vehicle = world.spawn_actor(
#     blueprint=vehicle_bp,
#     transform=world.get_map().get_spawn_points()[0])
# vehicle.set_autopilot(True)
# lidar = world.spawn_actor(
#     blueprint=lidar_bp,
#     transform=carla.Transform(carla.Location(x=1.0, z=1.8)),
#     attach_to=vehicle)

# lidar.listen(lambda data: sensor_callback(data))

# time.sleep(5.0)
# vehicle.destroy()
# lidar.destroy()

In [4]:
import gym
import gym_carla

import random
import datetime
import numpy as np
from tqdm import tqdm
from pathlib import Path
from agent_sac import Agent
from metrics import MetricLogger
from playground import PlayGround


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:


N_ACTIONS               = 2
INPUT_SHAPE             = (3, 256, 256)
REPLAY_SIZE             = 250
BATCH_SIZE              = 128
ALPHA                   = 0.00025
EPSILON                 = 1.0
EPSILON_DACAY           = 0.99999965 
EPISODES                = 100000
BURNIN                  = 10000
SAVE_EVERY              = 30000
UPDATE_EVERY            = 5000
TRAIN_EVERY             = 3
DISCOUNT                = 0.99



In [6]:
# def main():

params = {
    'number_of_vehicles': 20,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.05,  # time interval between two frames
    'discrete': False,  # whether to use discrete control space
    'discrete_acc': [-0.5, 0.0, 0.3, 0.6],  # discrete value of accelerations
    'discrete_steer': [-1.0, -0.5, 0.0, 0.5, 1.0],  # discrete value of steering angles
    'continuous_accel_range': [-1.0, 1.0],  # continuous acceleration range
    'continuous_steer_range': [-1.0, 1.0],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.model3.*',  # filter for defining ego vehicle
    'port': 2000,  # connection port
    'town': 'Town03',  # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 500,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 2.0,  # threshold for out of lane
    'desired_speed': 8,  # desired speed (m/s)
    'max_ego_spawn_times': 200,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    }

save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)
checkpoint = None
random.seed(1)
np.random.seed(1)

# Instantiate Environments
# Set gym-carla environment
env = gym.make('carla-v0', params=params)

tesla = Agent(
    alpha=ALPHA, 
    gamma=DISCOUNT, 
    epsilon=EPSILON, 
    tau=1,
    epsilon_decay=EPSILON_DACAY,
    max_size=REPLAY_SIZE, 
    burnin=BURNIN,
    input_shape=INPUT_SHAPE, 
    action_space=env.action_space,
    batch_size=BATCH_SIZE, 
    save_every=SAVE_EVERY, 
    update_every=UPDATE_EVERY,
    train_every=TRAIN_EVERY, 
    save_dir=save_dir, 
    checkpoint=checkpoint
)

logger = MetricLogger(save_dir=save_dir)


connecting to Carla server...
Carla server connected!


In [7]:

for epi in tqdm(range(1,EPISODES + 1), ascii=True, unit='episodes'):
    
    # Restarting episode - reset episode reward and step number
    episode_reward = 0
    state = env.reset()
    # print(state)
    print(state['camera'].shape)
    print(state['lidar'].shape)
    print(state['birdeye'].shape)
    # print(state['state'].shape)

    try:
    
        while True:
            action = tesla.act(state)
            next_state, reward, done, _ = env.step(action)
            tesla.cache(state, action, reward, next_state, done)
            updates = 0
            updates_per_step = 1 # model updates per simulator step (default: 1)
            for i in range(updates_per_step):
                loss, q_loss = tesla.learn(updates)
                logger.log_step(reward, loss, q_loss)
                updates += 1

            state = next_state
            # print(reward, loss)

            if done:
                break
    finally:
        #TODO: check what's up with this
        # env.destroy()
        env.reset()

    logger.log_episode()

    if epi % 20 == 0:
        logger.record(
            episode=epi,
            epsilon=tesla.epsilon,
            step=tesla.curr_step
        )
    
    #TODO: check what's up with this
    # if env.to_quit():
    #     env.set_synchronous_mode(False)
    #     break



  0%|          | 0/100000 [00:00<?, ?episodes/s]

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


  0%|          | 1/100000 [02:26<4068:10:06, 146.46s/episodes]

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


  0%|          | 2/100000 [04:30<3703:22:22, 133.32s/episodes]

...

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


  0%|          | 74/100000 [2:14:45<2668:18:50, 96.13s/episodes]

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


  0%|          | 75/100000 [2:16:21<2666:29:47, 96.07s/episodes]

(256, 256, 3)
(256, 256, 3)
(256, 256, 3)


In [ ]:

# if __name__ == '__main__':
#     main()
